In [2]:
import efg
import md
import sys
import numpy as np
import os
import nqr_parser as nqr
import datetime

################CHANGE THESE AS NEEDED##############
TEMP=223
TARGET_DIR='/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792931/scfs'
TARGET_SPECIES='N'
####################################################


os.chdir(TARGET_DIR) 
starttime=datetime.datetime.now()


print("Begin on {}".format(starttime.ctime()))
print("\nWorking dir:\t{}".format(os.getcwd())) 
print("""

TEMP:            {}
TARGET_DIR:      {}
TARGET_SPECIES:  {}
""".format(TEMP,TARGET_DIR,TARGET_SPECIES)
     )

print()




def theta_x(xi,yi,zi,xf,yf,zf):
    return np.arcsin(zf@yi)

def theta_y(xi,yi,zi,xf,yf,zf):
    thetax = theta_x(xi,yi,zi,xf,yf,zf)
    return zf@xi/np.cos(thetax)

def normalize(vector):
    return vector/np.linalg.norm(vector)

def ntheta_x(xi,yi,zi,xf,yf,zf):
    n=normalize
    xi,yi,zi,xf,yf,zf=n(xi),n(yi),n(zi),n(xf),n(yf),n(zf)
    return np.arcsin(zf@yi)

def ntheta_y(xi,yi,zi,xf,yf,zf):
    n=normalize
    xi,yi,zi,xf,yf,zf=n(xi),n(yi),n(zi),n(xf),n(yf),n(zf)
    thetax = theta_x(xi,yi,zi,xf,yf,zf)
    return zf@xi/np.cos(thetax)



ls = os.listdir('.') 
efgfiles = [ thing for thing in ls if thing.startswith('efg') and thing.endswith('out') ]
numfiles= len(efgfiles) 
print("There are {} efg files in this directory.".format(numfiles))
print()



efgs=[] 
skipped_files=[]
for i in range(numfiles): 
    infile='efg.{}.out'.format(i) 
    try:
        efgs.append(efg.Efg(infile))
    except FileNotFoundError:
        skipped_files.append(infile)
        #print('skipped file {}'.format(infile))

if len(skipped_files) != 0:
    print('{} files were skipped.'.format(len(skipped_files)))
    print('between {} and {}'.format(skipped_files[0], skipped_files[-1]))
    print('display variable "skipped_files" to see all files not opened.')
print("{} efg files opened without error.".format(len(efgs)))
print()



print("The indices of {} atoms are:".format(TARGET_SPECIES)) 
target_atom_labels=[] 
for label in efgs[0].atom_labels: 
    if TARGET_SPECIES in label: 
        new_index = efgs[0].atom_labels.index(label)+1 
        target_atom_labels.append(new_index) 
        sys.stdout.write(str(new_index)+' ') 

print()





av_theta_xs = [] 
av_theta_ys = [] 
av_theta_y2s = [] 
av_theta_x2s = [] 
cq_coef0s = [] 
cq_coef1s = [] 
eta_coefs = [] 
cq0s = [] 
cqprime0s = [] 
cqprime1s = [] 
eta0s = [] 
etaprimes = [] 
fq0s = [] 
fqprimes = []

for atnum in target_atom_labels: 
    efg0 = efgs[0] 
    specie=atnum 
    k=specie 
    print("\nWorking on atomic specie:\t{}".format(specie)) 
    
    
    this_nucleus=efg0.atom(k) 
    cq0=this_nucleus.cq 
    cq0s.append(cq0) 
    eta0=this_nucleus.eta
    eta0s.append(eta0)
    
    
    xi=efg0.atom(k).x
    yi=efg0.atom(k).y
    zi=efg0.atom(k).z



    theta_x_vec=[]
    theta_y_vec=[]


    print("Fetching the angles in file efg.{}.out")
    for i in range(1,901):
        try:
            nucleus=efgs[i].atom(k)
            nuc=nucleus
            xf,yf,zf=nuc.x,nuc.y,nuc.z
            args=(xi,yi,zi,xf,yf,zf)
            #print("thetaX{}\t".format(i),"\t", theta_x(xi,yi,zi,xf,yf,zf),"\t", "thetaY{}\t".format(i), theta_y(xi,yi,zi,xf,yf,zf))
            theta_x_vec.append(theta_x(*args))
            theta_y_vec.append(theta_y(*args))
            sys.stdout.write("{} \r".format(i))
        except IndexError:
            sys.stdout.write("skipped:{}\n".format(i))
        
  
                 # theta_x squareds and theta_y squareds
    theta_x2_vec=[angle**2 for angle in theta_x_vec]
    theta_y2_vec=[angle**2 for angle in theta_y_vec]

    av=np.average
    av_theta_x  = av(theta_x_vec)
    av_theta_y  = av(theta_y_vec)
    av_theta_x2 = av(theta_x2_vec)
    av_theta_y2 = av(theta_y2_vec)

    av_theta_xs.append(av_theta_x)
    av_theta_ys.append(av_theta_y)
    av_theta_x2s.append(av_theta_x2)
    av_theta_y2s.append(av_theta_y2)

    print("\nReport for atomic specie:\t{}".format(k))
    print("Working Dir:\t{}".format(os.getcwd()))
    print("<tx>:\t",av_theta_x,"\t<ty>:\t",   av_theta_y,"\n",
      "<tx^2>\t:",av_theta_x2,"\t<ty^2>:", av_theta_y2,"\n")

    print("cq0=\t{}".format(cq0))
    print("eta0=\t{}".format(eta0))

    cq_coef0 = 1 - 3/2*(av_theta_x2 + av_theta_y2)
    cq_coef0s.append(cq_coef0)
    cq_coef1 = 1 - 3/2*(av_theta_x2 + av_theta_y2 - 1/2*eta0*(av_theta_x2 - av_theta_y2))
    cq_coef1s.append(cq_coef1)

    
    
    print("==================================")
    print("target:\t"+TARGET_DIR)
    print("cwd:\t"+os.getcwd())
    print("T={}K, nuclear site: {}  =".format(TEMP,specie))
    print("==================================")


    print("1-3/2(<tx^2>+<ty^2>)\t\t={}".format(cq_coef0))
    print("1-3/2(<tx^2>+<ty^2>-1/2eta0*(<tx^2>-<ty^2>)\t={}".format(cq_coef1))
    print()


    cqprime0=cq0*cq_coef0
    cqprime0s.append(cqprime0)
    cqprime1=cq0*cq_coef1
    cqprime1s.append(cqprime1)


    print("Cq0:\t\t{}".format(cq0))
    print("Cq' from coefficient 1-3/2(<>+<>):\t\t{}".format(cqprime0))
    print("Cq' from coefficient 1-3/2(<>+<>)+1/2eta(<>-<>)):\t{}".format(cqprime1))
    nuclear_spin=1
    freq_function=nqr.f1
    print(
    "Computing NQR frequencies for spin:\t{}\nusing function {}\tand simple coefficient".format(nuclear_spin, freq_function)
    )
    print("WARNING:\t eta not adjusted.  Using eta0.")
    fq=freq_function(cqprime0,eta0)
    fq0s.append(fq)
    print("fqs={}".format(freq_function(cqprime0, eta0)))
    print()
    print()

print("##################################"*2)
print("##################################"*2)
print()
print("==================================")
print("          FINAL REPORT T={}K      ".format(TEMP))
print("cwd:\t{}".format(os.getcwd()))
print("==================================")
print("""
target_atom_labels=\t{}\n
av_theta_xs=\t{}\n    
av_theta_ys=\t{}\n  
av_theta_y2s=\t{}\n
av_theta_x2s=\t{}\n 
cq_coef0s=\t{}\n
cq_coef1s=\t{}\n
eta_coefs=\t{}\n
cq0s=\t{}\n
cqprime0s=\t{}\n
cqprime1s=\t{}\n 
eta0s=\t{}\n
etaprimes=\t{}\n
fq0s=\t{}\n 
fqprimes=\t{}\n


    """.format(
    target_atom_labels,
    av_theta_xs,
    av_theta_ys, 
    av_theta_y2s, 
    av_theta_x2s, 
    cq_coef0s, 
    cq_coef1s, 
    eta_coefs, 
    cq0s, 
    cqprime0s, 
    cqprime1s, 
    eta0s, 
    etaprimes, 
    fq0s, 
    fqprimes))





print("cq_coef0s\n---------")
for item in target_atom_labels:
    print(item, cq_coef0s[target_atom_labels.index(item)])
print()


print("cq_coef1s\n---------")
for item in target_atom_labels:
    print(item, cq_coef1s[target_atom_labels.index(item)])
print()


print("cq0s\n---------")
for item in target_atom_labels:
    print(item, cq0s[target_atom_labels.index(item)])
print()


print("cqprime0s\n---------")
for item in target_atom_labels:
    print(item, cqprime0s[target_atom_labels.index(item)])
print()


print("cqprime1s\n---------")
for item in target_atom_labels:
    print(item, cqprime1s[target_atom_labels.index(item)])
print()


print("eta0s\n---------")
for item in target_atom_labels:
    print(item, eta0s[target_atom_labels.index(item)])
print()





endtime=datetime.datetime.now()
runtime=endtime-starttime
runtime_s=runtime.total_seconds()

print("DONE\truntime is {} seconds".format(str(runtime_s)))
print(endtime.ctime())
print("Goodbye.")
print()




Begin on Mon Feb 19 14:44:43 2018

Working dir:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792931/scfs


TEMP:            223
TARGET_DIR:      /Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792931/scfs
TARGET_SPECIES:  N


There are 901 efg files in this directory.

901 efg files opened without error.

The indices of N atoms are:
1 3 4 6 23 25 26 28 37 39 40 42 51 53 54 56 

Working on atomic specie:	1
Fetching the angles in file efg.{}.out
skipped:851
skipped:852
skipped:853
skipped:854
skipped:855
skipped:856
skipped:857
skipped:858
skipped:859
skipped:860
skipped:861
skipped:862
skipped:863
skipped:864
skipped:865
skipped:866
skipped:867
skipped:868
skipped:869
skipped:870
skipped:871
skipped:872
skipped:873
skipped:874
skipped:875
skipped:876
skipped:877
skipped:878
skipped:879
skipped:880
skipped:881
skipped:882
skipped:883
skipped:884
skipped:885
skipped:886
skipped:887
skipped:888
skipped:889
skipped:890
skipped:891
skipped:892
skipped:893
skipped:894
skipped:895
skipp

Fetching the angles in file efg.{}.out
skipped:851
skipped:852
skipped:853
skipped:854
skipped:855
skipped:856
skipped:857
skipped:858
skipped:859
skipped:860
skipped:861
skipped:862
skipped:863
skipped:864
skipped:865
skipped:866
skipped:867
skipped:868
skipped:869
skipped:870
skipped:871
skipped:872
skipped:873
skipped:874
skipped:875
skipped:876
skipped:877
skipped:878
skipped:879
skipped:880
skipped:881
skipped:882
skipped:883
skipped:884
skipped:885
skipped:886
skipped:887
skipped:888
skipped:889
skipped:890
skipped:891
skipped:892
skipped:893
skipped:894
skipped:895
skipped:896
skipped:897
skipped:898
skipped:899
skipped:900

Report for atomic specie:	25
Working Dir:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792931/scfs
<tx>:	 0.00522650966264 	<ty>:	 -0.00812436019024 
 <tx^2>	: 0.00471716821147 	<ty^2>: 0.00242981805494 

cq0=	-6.095
eta0=	0.47165
target:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792931/scfs
cwd:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6

Report for atomic specie:	40
Working Dir:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792931/scfs
<tx>:	 -0.00783850349183 	<ty>:	 -0.00398278764173 
 <tx^2>	: 0.0575555470653 	<ty^2>: 0.0260308540694 

cq0=	-0.7202
eta0=	0.64108
target:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792931/scfs
cwd:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792931/scfs
T=223K, nuclear site: 40  =
1-3/2(<tx^2>+<ty^2>)		=0.8746203982980068
1-3/2(<tx^2>+<ty^2>-1/2eta0*(<tx^2>-<ty^2>)	=0.8897777859373583

Cq0:		-0.7202
Cq' from coefficient 1-3/2(<>+<>):		-0.6299016108542245
Cq' from coefficient 1-3/2(<>+<>)+1/2eta(<>-<>)):	-0.6408179614320855
Computing NQR frequencies for spin:	1
using function <function f1 at 0x107d9fe18>	and simple coefficient
fqs=[-0.20190866 -0.37147188 -0.57338054]



Working on atomic specie:	42
Fetching the angles in file efg.{}.out
skipped:851
skipped:852
skipped:853
skipped:854
skipped:855
skipped:856
skipped:857
skipped:858
skipped:859
skipped:860
skipped:861
sk

In [1]:
import efg
import md
import sys
import numpy as np
import os
import nqr_parser as nqr
import datetime

target_atom_labels=	[1, 3, 4, 6, 23, 25, 26, 28, 37, 39, 40, 42, 51, 53, 54, 56]

av_theta_xs=	[-0.0074969194066950877, 0.0013565016919028347, 0.01860370232894331, 0.01411222573569982, -0.014358371926024236, 0.0052265096626419741, -0.0092767755042074442, 0.011041491512496924, 0.020436193401444966, 0.023660404879824036, -0.007838503491833293, -0.016030481289655462, -0.020154873992586191, -0.0019720681844344022, -0.0087921365579179227, 0.015573744026243378]
    
av_theta_ys=	[-0.001491688508229571, -0.0027115679749485796, -0.01439269149302657, -0.0038380900092672395, 0.0012778461351861062, -0.0081243601902447286, -0.0058349053122594115, -0.0064115789314661449, -0.0086124027854763089, -0.0085496468512345856, -0.0039827876417324075, 0.00145070198615622, 0.000814166311416736, -0.0055292050339317807, -0.011279858516833356, 0.0014819863962864739]
  
av_theta_y2s=	[0.0030952435895492294, 0.018145508497458628, 0.0023250290544682148, 0.012772513892434117, 0.015261131692525983, 0.0024298180549448848, 0.019196257133641301, 0.0036739020053258142, 0.020277126966272811, 0.0027255796143571785, 0.026030854069388504, 0.0031315509947800134, 0.0077466985189401424, 0.0032832250363852102, 0.018341079795736077, 0.0035303664167893596]

av_theta_x2s=	[0.002763841792080125, 0.034097007262718297, 0.0032653061292592218, 0.050033679731203241, 0.054182519772639937, 0.0047171682114675916, 0.056297051596054408, 0.0019669471142757448, 0.042336793621296533, 0.00432145049171979, 0.057555547065273653, 0.0041115391863721122, 0.052429649277433862, 0.0037366343906113267, 0.042324964096523289, 0.0039731834377277394]
 
cq_coef0s=	[0.99121137192755593, 0.92163622635973463, 0.99161449722440886, 0.90579070956454399, 0.89583452280225107, 0.98927952060038127, 0.88676003690545646, 0.9915387263205977, 0.90607911911864603, 0.98942945484088451, 0.87462039829800675, 0.98913536472827179, 0.90973547830543899, 0.98947021085950515, 0.90900093416161099, 0.9887446752182244]

cq_coef1s=	[0.99108520477774598, 0.92930586648105917, 0.99194710848615275, 0.92177407295691316, 0.91253033873656697, 0.99008864212637426, 0.90459707861113692, 0.99088887579141327, 0.9155417234286517, 0.98999397421536561, 0.88977778593735835, 0.98950845358272232, 0.92890278857017938, 0.98963059875094572, 0.92053172617661116, 0.98891325897922322]

eta_coefs=	[]

cq0s=	[-6.225, -0.7202, -6.095, -0.7898, -0.7898, -6.095, -0.7202, -6.225, -0.7898, -6.095, -0.7202, -6.225, -0.7898, -6.095, -0.7202, -6.225]

cqprime0s=	[-6.1702907902490356, -0.66376241022428084, -6.0438903605827718, -0.71539350241407684, -0.70753010610921785, -6.0296586780593238, -0.63864457857930967, -6.1723285713457203, -0.71562128827990656, -6.0305725272551909, -0.62990161085422447, -6.1573676454334914, -0.71850908076563569, -6.0308209351886832, -0.65466247278319223, -6.1549356032334464]

cqprime1s=	[-6.1695053997414684, -0.66928608503965881, -6.0459176262231011, -0.72801716282136997, -0.7207164615341406, -6.0345902737602506, -0.65149081601574077, -6.1682832518015474, -0.72309485316394906, -6.0340132728426532, -0.64081796143208547, -6.1596901235524459, -0.73364742241272762, -6.0317984993870137, -0.66296694919239529, -6.1559850371456646]
 
eta0s=	[0.50761, 0.64108, 0.47165, 0.57194, 0.57195, 0.47165, 0.64103, 0.50761, 0.57194, 0.47165, 0.64108, 0.50761, 0.57195, 0.47165, 0.64103, 0.50761]

etaprimes=	[]

fq0s=  [np.array([-1.56605065, -3.84469277, -5.41074342]),np.array([-0.2127624 , -0.39144061, -0.60420301]),np.array([-1.42530044, -3.82026755, -5.24556799]),np.array([-0.20458108, -0.43425459, -0.63883567]),np.array([-0.20233592, -0.42947962, -0.63181554]),np.array([-1.42194426, -3.81127188, -5.23321614]),np.array([-0.20469517, -0.37663585, -0.58133102]),np.array([-1.56656785, -3.8459625 , -5.41253036]),np.array([-0.20464622, -0.43439286, -0.63903908]),np.array([-1.42215977, -3.81184951, -5.23400928]),np.array([-0.20190866, -0.37147188, -0.57338054]),np.array([-1.5627707 , -3.83664039, -5.39941108]),np.array([-0.20547563, -0.43614399, -0.64161963]),np.array([-1.42221835, -3.81200653, -5.23422487]),np.array([-0.20982914, -0.38608228, -0.59591143]),np.array([-1.56215343, -3.83512499, -5.39727842])]

fqprimes=	[]

TEMP=223
TARGET_DIR='/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792931'
os.chdir(TARGET_DIR)
os.getcwd()
x=md.Md('md.in','md.out')
vol=x.cell_volume
print(TARGET_DIR,"\n","-"*len(TARGET_DIR),"\n")

def get_inds(target, inds):
    result = []
    for ind in inds:
        result.append(target[ind])
    return result

axial_inds = [ 0,  7, 11, 15]
equit_inds = [ 2,  5,  9, 13]
ax         = axial_inds
eq         = equit_inds

axial_cq0s = get_inds(cq0s, ax)
axial_eta0s= get_inds(eta0s, ax)
axial_k0s  = get_inds(cq_coef0s, ax)
axial_k1s  = get_inds(cq_coef1s, ax)
axial_cq   = np.average(get_inds(cq0s, ax))
axial_eta  = np.average(get_inds(eta0s, ax))
axial_k0   = np.average(get_inds(cq_coef0s, ax))
axial_k1   = np.average(get_inds(cq_coef1s, ax))


equit_cq0s = get_inds(cq0s, eq)
equit_eta0s= get_inds(eta0s, eq)
equit_k0s  = get_inds(cq_coef0s, eq)
equit_k1s  = get_inds(cq_coef1s, eq)
equit_cq   = np.average(get_inds(cq0s, eq))
equit_eta  = np.average(get_inds(eta0s, eq))
equit_k0   = np.average(get_inds(cq_coef0s, eq))
equit_k1   = np.average(get_inds(cq_coef1s, eq))



print("cell volume:", vol,"\n")
print("AXIAL","\n","-"*len(TARGET_DIR))
print("avg Cq0:", np.average(axial_cq0s),"\t",axial_cq)
print("avg eta0:", np.average(axial_eta0s),"\t",axial_eta)
print("avg coef0:",np.average(axial_k0s),"\t",axial_k0)
print("avg coef1:",np.average(axial_k1s),"\t",axial_k1)
print("using coef0:",axial_k0*axial_cq)
print("using coef1:",axial_k1*axial_cq)


ffunc=nqr.f1
fraw=ffunc(axial_cq,axial_eta)
f0=ffunc(axial_k0*axial_cq,axial_eta)
f1=ffunc(axial_k1*axial_cq,axial_eta)
exptcq=6.029
expteta=0.5205
exptfq=[5.306,       3.737]
print("expt cq:\t", exptcq)
print("raw freq:\t",list(reversed(fraw))[:-1])

print("expt fq:\t", exptfq)
print("adjusted f0s:\t",list(reversed(f0))[:-1])
print("adjusted f1s:\t",list(reversed(f1))[:-1])
print("\n\n")

print("EQUITORIAL","\n","-"*len(TARGET_DIR))
print("avg Cq0:", np.average(equit_cq0s),"\t",equit_cq)
print("avg eta0:", np.average(equit_eta0s),"\t",equit_eta)
print("avg coef0:",np.average(equit_k0s),"\t",equit_k0)
print("avg coef1:",np.average(equit_k1s),"\t",equit_k1)
print("using coef0:",equit_k0*equit_cq)
print("using coef1:",equit_k1*equit_cq)


ffunc=nqr.f1
fraw=ffunc(equit_cq,equit_eta)
f0=ffunc(equit_k0*equit_cq,equit_eta)
f1=ffunc(equit_k1*equit_cq,equit_eta)
exptcq=5.795
expteta=0.4980
exptfq=[5.068,       3.625]
print("expt cq:\t", exptcq)
print("raw freq:\t",list(reversed(fraw))[:-1])

print("expt fq:\t", exptfq)
print("adjusted f0s:\t",list(reversed(f0))[:-1])
print("adjusted f1s:\t",list(reversed(f1))[:-1])

/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792931 
 -------------------------------------------------------- 

cell volume: 512.446954146 

AXIAL 
 --------------------------------------------------------
avg Cq0: -6.225 	 -6.225
avg eta0: 0.50761 	 0.50761
avg coef0: 0.990157534549 	 0.990157534549
avg coef1: 0.990098948283 	 0.990098948283
using coef0: -6.16373065257
using coef1: -6.16336595306
expt cq:	 6.029
raw freq:	 [-5.4587180624999991, -3.8787819374999994]
expt fq:	 [5.306, 3.737]
adjusted f0s:	 [-5.4049908185612523, -3.8406051602868843]
adjusted f1s:	 [-5.4046710126534441, -3.8403779169369789]



EQUITORIAL 
 --------------------------------------------------------
avg Cq0: -6.095 	 -6.095
avg eta0: 0.47165 	 0.47165
avg coef0: 0.989948420881 	 0.989948420881
avg coef1: 0.990415080895 	 0.990415080895
using coef0: -6.03373562527
using coef1: -6.03657991805
expt cq:	 5.795
raw freq:	 [-5.2899266875000004, -3.8525733125000001]
expt fq:	 [5.068, 3.625]
adjusted f0s:	 [-5.